In [1]:
import requests
import io

from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))
c.head(3)

,date,location,new_cases,new_deaths,total_cases,total_deaths
0,2019-12-31,Afghanistan,0,0,0,0
1,2020-01-01,Afghanistan,0,0,0,0
2,2020-01-02,Afghanistan,0,0,0,0


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv")

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-04-28,France,130141.0,0.0,23715.0,0.0,0.0,0.0
0,2020-04-28,China,84042.0,0.0,4670.0,0.0,0.0,0.0
0,2020-04-28,Italy,201633.0,0.0,27211.0,0.0,0.0,0.0
0,2020-04-28,Spain,210679.0,0.0,23372.0,0.0,0.0,0.0
0,2020-04-28,United States,1012803.0,0.0,57786.0,0.0,0.0,0.0
0,2020-04-28,World,3068693.0,0.0,213200.0,0.0,0.0,0.0
0,2020-04-28,United Kingdom,161420.0,0.0,21356.0,0.0,0.0,0.0
0,2020-04-28,Germany,157018.0,0.0,6046.0,0.0,0.0,0.0
0,2020-04-28,Iran,92576.0,0.0,5869.0,0.0,0.0,0.0
0,2020-04-28,Turkey,113857.0,0.0,2988.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")

In [6]:
## concatenate all df in 1
frames = [follow_df1,follow_df2,follow_df3,follow_df4,follow_df5,follow_df6,follow_df7,follow_df8,follow_df9,follow_df10]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-04-29,France,130141.0,126835.0,23715.0,23660.0,3306.0,55.0
0,2020-04-29,China,84042.0,83940.0,4670.0,4637.0,102.0,33.0
0,2020-04-29,Italy,201633.0,201505.0,27211.0,27359.0,128.0,-148.0
0,2020-04-28,Spain,210679.0,210773.0,23372.0,23822.0,-94.0,-450.0
0,2020-04-29,United States,1012803.0,1012583.0,57786.0,58355.0,220.0,-569.0
0,2020-04-29,World,3068693.0,3052370.0,213200.0,216563.0,16323.0,-3363.0
0,2020-04-29,United Kingdom,161420.0,161145.0,21356.0,21678.0,275.0,-322.0
0,2020-04-29,Germany,157018.0,157641.0,6046.0,6115.0,-623.0,-69.0
0,2020-04-29,Iran,92576.0,92584.0,5869.0,5877.0,-8.0,-8.0
0,2020-04-29,Turkey,113857.0,114653.0,2988.0,2992.0,-796.0,-4.0


In [7]:
## dump dataset for use it in other nb
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")